# Distributed Computing: `Distributed` standard library

**What**
* **single Julia process -> multiple Julia processes** that coordinate to perform certain computations

**Why**
* Scaling things up: run computations on multiple CPU cores, potentially even on different machines, e.g. nodes of a supercomputer or a local cluster of desktop machines.
* Effectively increase your memory: process a large dataset, which wouldn't fit into local memory, in parallel across multiple machines with separate dedicated RAM.

**Julia provides two fundamental implementations and paradigms**
* Julia's built-in [`Distributed` standard library](https://docs.julialang.org/en/v1/stdlib/Distributed/)
  * master-worker model
* [Message Passing Interface (MPI)](https://www.mpi-forum.org/) through [MPI.jl](https://github.com/JuliaParallel/MPI.jl)
  * Single Program Multiple Data (SPMD)
  
The focus of this notebook is on the **`Distributed` standard library.**

## `Distributed` standard library

Julia's `Distributed` follows a master-worker paradigm for its native distributed parallelism: **One master process coordinates all the worker processes, which perform the actual computations.**

In [1]:
using Distributed

In [2]:
nprocs()

1

In [3]:
nworkers() # the master is considered a worker as long as there are no real workers

1

To increase the number of workers, i.e. Julia processes, from within a Julia session we can dynamically call **`addprocs`**.

Alternatively, when starting Julia from the command line, one can use the `-p` option up front. Example,

```
julia -p 4
```

will start Julia with 5 processes, 1 master and 4 workers.

In [4]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

Every process has a Julia internal `pid` (process id). The master is always 1. You can get the workers pids from `workers()`.

In [5]:
workers()

4-element Vector{Int64}:
 2
 3
 4
 5

Note that the 4 worker's pids aren't necessarily 2, 3, 4 and 5 and one shouldn't rely on those literal values. Let's remove the processes and add them once more.

In [6]:
rmprocs(workers())

Task (done) @0x0000000119edacb0

In [7]:
nworkers() # only the master is left

1

In [8]:
addprocs(4)

4-element Vector{Int64}:
 6
 7
 8
 9

In [9]:
workers()

4-element Vector{Int64}:
 6
 7
 8
 9

### One master to rule them all - `@spawn`, `@spawnat`, `@fetch`, `@fetchfrom`, `@everywhere`...

To execute commands and start computations on workers we can use the following macros

* `@spawn`: run a command or a code block on any worker and return a `Future` to it's result. It's basically a version of `@async` for remote processes.
* `@spawnat`: same as `@spawn` but one can choose a specific worker by providing its pid.

**Example:** Let's say we would like to generate a random matrix on one of the workers.

In [40]:
@spawn 3+3 # similar to @async

Future(8, 1, 24, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (3, 4294967296, 4520084976)), nothing)

In [41]:
result = @spawn 3+3

Future(9, 1, 25, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 4964320784, 107374182406)), nothing)

In [42]:
fetch(result)

6

Because the combination of spawning at fetching is so common, there is `@fetch` which combines them.

In [44]:
@fetch 3+3

6

In [46]:
@fetch rand(3,3)

3×3 Matrix{Float64}:
 0.0725835  0.145207   0.582114
 0.69949    0.0627775  0.721059
 0.0933021  0.660484   0.235878

Which worker did the work?

In [51]:
@fetch begin
    println(myid());
    3+3
end

      From worker 8:	8


6

Using `@spawnat` and `@fetchfrom` we can delegate the work to a specific worker.

In [49]:
@fetchfrom 7 begin
    println(myid());
    3+3
end

      From worker 7:	7


6

We can use `@sync` as a blocker to wait for all workers to complete their tasks.

In [53]:
@sync begin
    pids = workers()
    @spawn (sleep(2); println("Today is reverse day!"))
    @spawn (sleep(1); println(" class!"))
    @spawn println("Hello")
end;
println("Done!")

      From worker 7:	Hello
      From worker 6:	 class!
      From worker 9:	Today is reverse day!
Done!


Ok, now that we understood all that, let's delegate a *complicated* calculation

In [56]:
using Random

function complicated_calculation()
    sleep(1) # so complex that it takes a long time :)
    randexp(5)
end

@fetch complicated_calculation()

RemoteException: On worker 9:
UndefVarError: #complicated_calculation not defined
Stacktrace:
  [1] deserialize_datatype
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Serialization/src/Serialization.jl:1364
  [2] handle_deserialize
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Serialization/src/Serialization.jl:866
  [3] deserialize
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Serialization/src/Serialization.jl:813
  [4] handle_deserialize
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Serialization/src/Serialization.jl:873
  [5] deserialize
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Serialization/src/Serialization.jl:813 [inlined]
  [6] deserialize_global_from_main
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Distributed/src/clusterserialize.jl:160
  [7] #5
    @ ~/.julia/juliaup/julia-1.8.0+0.x64/share/julia/stdlib/v1.8/Distributed/src/clusterserialize.jl:72 [inlined]

What happened?

**Every worker is a separate Julia process.** (Think of having multiple Julia REPLs open at once.)

We only defined `complicated_calculation()` on the master process. The function doesn't exist on any of the workers yet.

The macro `@everywhere` allows us to perform steps on all processes (master and worker). This is particularly useful for loading packages and functions definitions etc.

In [57]:
@everywhere begin # execute this block on all workers
    using Random
    
    function complicated_calculation()
        sleep(1)
        randexp(5) # lives in Random
    end
end

In [58]:
@fetch complicated_calculation()

5-element Vector{Float64}:
 0.824451959101157
 1.3304595721548624
 0.12687671104385195
 1.6127992535773963
 1.616419007086614

### Data movement

There is a crucial difference between the following two pieces of code. Can you guess what it is? (without reading on 😉)

In [59]:
function method1()
    A = rand(100,100)
    B = rand(100,100)
    C = @fetch A^2 * B^2
end

method1 (generic function with 1 method)

In [60]:
function method2()
    C = @fetch rand(100,100)^2 * rand(100,100)^2
end

method2 (generic function with 1 method)

Let's benchmark them.

In [61]:
using BenchmarkTools
@btime method1();
@btime method2();

  646.503 μs (98 allocations: 237.99 KiB)
  414.795 μs (68 allocations: 80.95 KiB)


Method 1 is slower, because `A` and `B` are created on the master process, transferred to a worker, and squared and multiplied on the worker process before the result is finally transferred back to the master.

Method 2, on the other hand, creates, squares, and multiplies the random matrix all on the work process and only submits the result to the master.

Hence, `method1` is **transferring 3x as much data** between the master and the worker!

**Efficient data movement is crucial for efficient parallel computing!**

In this toy example, it's rather easy to identify the faster method.

In a real program, however, understanding data movement does require more thought and likely some measurement.

For example, if the first process needs matrix `A` in a follow-up computation then the first method might be better in this case. Or, if computing `A` is expensive and only the current process has it, then moving it to another process might be unavoidable.

#### Computer latency at a human scale

To understand why thinking about data is important it's instructive to look at the time scales involved in data access.

<img src="../imgs/latency_human_scales.png" width=900px>

(taken from https://www.prowesscorp.com/computer-latency-at-a-human-scale/)

#### Avoid globals (once more)

In [63]:
myglobal = 4

4

In [64]:
function whohas(s::String)
    @everywhere begin
        var = Symbol($s)
        if isdefined(Main, var)
            println("$var exists.")
        else
            println("Doesn't exist.")
        end
    end
    nothing
end

whohas (generic function with 1 method)

In [65]:
whohas("myglobal")

myglobal exists.
      From worker 9:	Doesn't exist.
      From worker 7:	Doesn't exist.
      From worker 6:	Doesn't exist.
      From worker 8:	Doesn't exist.


In [66]:
@fetchfrom 6 myglobal+2

6

In [67]:
whohas("myglobal")

myglobal exists.
      From worker 9:	Doesn't exist.
      From worker 7:	Doesn't exist.
      From worker 8:	Doesn't exist.
      From worker 6:	myglobal exists.


Globals get copied to workers and continue to exist as globals even after the call.

This could lead to **memory accumulation** if many globals are used (just as it would in a single Julia session).

It's better to avoid them.

#### Explicit data movement: `Channel` and `RemoteChannel`

Implement communication between tasks. Functions: `put!`, `take!`, `fetch`, `isready` and `wait`.

In [69]:
ch = Channel{Int}(5) # a channel that can hold up to 5 integers

Channel{Int64}(5) (empty)

In [100]:
isready(ch) # something in the channel?

false

In [85]:
put!(ch, 3)

3

In [101]:
isready(ch)

false

In [ ]:
3+3

In [102]:
fetch(ch)

In [89]:
take!(ch)

4

In [83]:
isready(ch)

false

In [86]:
put!(ch, 4)

4

In [76]:
fetch(ch)

4

In [77]:
take!(ch)

4

**Be careful**, `take!` and `put!` are blocking if the channel is empty or full!

In [92]:
isready(ch)

false

In [ ]:
# take!(ch) if we execute this, while isready(ch) == false, the current Julia session will hang.

##### `RemoteChannel`

* A `Channel` is local to a process. Worker 2 cannot directly refer to a `Channel` on worker 3 and vice-versa.


* A `RemoteChannel`, however, can put and take values across workers. A `RemoteChannel` can be thought of as a handle to a `Channel`.


* Any process with a reference to a `RemoteChannel` can put and take items from the channel. Data is automatically sent to (or retrieved from) the process a `RemoteChannel` is associated with.


* The process id, pid, associated with a `RemoteChannel` identifies the process where the backing store, i.e., the backing Channel exists.

In [91]:
nworkers()

4

In [ ]:
addprocs(4)

In [97]:
function do_something()
    rc = RemoteChannel(()->Channel{Int}(10)) # lives on the master
    @sync for p in workers()
        @spawnat p put!(rc, myid())
    end
    rc
end

r = do_something()

RemoteChannel{Channel{Int64}}(1, 1, 8804)

In [98]:
isready(r)

true

In [99]:
while isready(r)
    @show take!(r)
end

take!(r) = 9
take!(r) = 8
take!(r) = 7
take!(r) = 6


The ecosystem also contains a couple of tools, that make data transfer even simpler. See for example [ParallelDataTransfer.jl](https://github.com/ChrisRackauckas/ParallelDataTransfer.jl/).

## High-level tools: `@distributed` and `pmap`

So far we have seen some of the fundamental building blocks for distributed computing in Julia. However, in practice, one wants to think as little as possible about how to distribute the work and explicitly spawn tasks.

Fortunately, many useful parallel computations do not require (much) data movement at all. A common example is a direct Monte Carlo simulation, where multiple processes can handle independent simulation trials simultaneously. (We'll get to that later in the exercises!)

Julia provides **high-level convenience** tools to
 * parallelize loops ([**`@distributed`**](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.@distributed)) and
 * apply a function to all elements of a collection ([**`pmap`**](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.pmap))


#### Distributed loops (`@distributed`)

In [2]:
using Distributed, BenchmarkTools; rmprocs(workers()); addprocs(4); nworkers()

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-x64-6.0/build/default-macmini-x64-6-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Distributed/src/cluster.jl:1048


4

#### Example: Reduction

Task: Counting heads in a series of coin tosses.

In [51]:
function count_heads_loop(n)
    c = 0
    for i = 1:n
        c += rand(Bool)
    end
    return c
end

N = 200_000_000
@btime count_heads_loop($N);

  322.219 ms (0 allocations: 0 bytes)


Note that these kinds of computations are called **reductions** (with `+` being the **reducer function**).

In [52]:
count_heads_reduce(n) = mapreduce(i -> rand(Bool), +, 1:n)
@btime count_heads_reduce($N)

  326.941 ms (0 allocations: 0 bytes)


100004576

In [56]:
function count_heads_distributed_loop(n)
    c = @distributed (+) for i in 1:n
        Int(rand(Bool))
    end
    return c
end

count_heads_distributed_loop (generic function with 1 method)

In [54]:
@btime count_heads_distributed_loop($N);

  81.147 ms (302 allocations: 12.55 KiB)


The distributed version is about **4x faster**, which is all we could hope for.

With `@distributed` the work is **evenly distributed** between the workers.

In [16]:
function count_heads_distributed_verbose(n)
    c = @distributed (+) for i in 1:n
        x = Int(rand(Bool))
        println(x);
        x
    end
    c
end

count_heads_distributed_verbose(8);

      From worker 3:	0
      From worker 2:	0
      From worker 4:	0
      From worker 5:	1
      From worker 5:	0
      From worker 3:	1
      From worker 2:	1
      From worker 4:	0


However, by using `@distributed` we let Julia decide how to split up the work and can't control it ourselves.

#### A common mistake

In [122]:
function g(n)
    a = 0
    @distributed (+) for i in 1:n
        a += 1
    end
    a
end

a = g(10);

What do you expect the value of `a` to be?

In [123]:
a

0

#### Example: `SharedArray`s

Apart from `@distributed (reducer) ...` there also is a `@distributed for ...` form. The latter is **non-blocking** and returns a `Task`. (You can think of it as a distributed version of `@spawn` for all the iterations.)

However, since the loop body will be executed on different processes, one must be careful to operate on data structures that are available on all processes (similar to the mistake highlighted above).

In [95]:
function square_broken()
    A = collect(1:10)
    @sync @distributed for i in eachindex(A)
        A[i] = A[i]^2
    end
    return A
end

square_broken (generic function with 1 method)

In [96]:
square_broken()

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

To actually make all processes operate on the same array, one can use a `SharedArray`. For this to work, the **processes need to live on the same host**.

In [97]:
@everywhere using SharedArrays # must be loaded everywhere

In [98]:
A = rand(3,2)

3×2 Matrix{Float64}:
 0.538336  0.0201671
 0.212669  0.888775
 0.405967  0.313966

In [99]:
S = SharedArray(A)

3×2 SharedMatrix{Float64}:
 0.538336  0.0201671
 0.212669  0.888775
 0.405967  0.313966

In [101]:
function square!(X)
    for i in eachindex(X)
        sleep(0.001) # mimicing some computational cost
        X[i] = X[i]^2
    end
end

function square_distributed!(X)
    @sync @distributed for i in eachindex(X)
        sleep(0.001) # mimicing some computational cost
        X[i] = X[i]^2
    end
end

A = rand(10,10)
S = SharedArray(A)

@btime square!(A);
@btime square_distributed!($S);

  229.457 ms (508 allocations: 15.31 KiB)
  58.816 ms (574 allocations: 26.09 KiB)


### Parallel map: `pmap`

The `square!` functions above are typical `map` operations where a function `f` is applied to all elements of a collection.

In [103]:
map(x->x^2, 1:10)

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

Such a pattern can be parallelized in Julia via the high-level function `pmap` ("parallel map").

#### Example: Singular values of multiple matrices

In [105]:
using Distributed, BenchmarkTools; rmprocs(workers()); addprocs(4); nworkers()

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-x64-6.0/build/default-macmini-x64-6-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Distributed/src/cluster.jl:1048


4

In [106]:
@everywhere using LinearAlgebra

M = Matrix{Float64}[rand(200,200) for i = 1:10];

In [108]:
svdvals(rand(2,2))

2-element Vector{Float64}:
 1.567295814264191
 0.05638577289583601

In [110]:
map(svdvals, M)

10-element Vector{Vector{Float64}}:
 [99.6378139397788, 7.953440321631967, 7.89163500923745, 7.794278173305783, 7.601075754241636, 7.532778071932169, 7.495234488220123, 7.37095663387068, 7.290094762057224, 7.24038507051588  …  0.2577599248490027, 0.24168950803061742, 0.19460642318981866, 0.17353097393282843, 0.16126794350742069, 0.14854630524019563, 0.12259385086810454, 0.05567251849314098, 0.028137990789380413, 0.004019967434655436]
 [99.97491934942695, 8.136311451491352, 7.9045359058287215, 7.847918834879346, 7.740466115796857, 7.596100441052067, 7.5252931307955215, 7.422301114226823, 7.278904160097977, 7.154623741055865  …  0.28764020300987836, 0.23672411802597415, 0.22617107993934113, 0.17734897787737702, 0.1640599747511493, 0.10738905216166718, 0.09961959432527073, 0.06895712121746422, 0.04113860155120967, 0.019051549507549954]
 [100.54302411474777, 8.066270701024377, 7.816097994710581, 7.740430872496169, 7.672445958918696, 7.601081020110799, 7.497556443647828, 7.40728999820195, 7

In [111]:
pmap(svdvals, M)

10-element Vector{Vector{Float64}}:
 [99.6378139397788, 7.953440321631967, 7.891635009237456, 7.794278173305781, 7.601075754241633, 7.532778071932167, 7.495234488220115, 7.370956633870673, 7.290094762057221, 7.240385070515878  …  0.2577599248490024, 0.2416895080306173, 0.1946064231898194, 0.17353097393282851, 0.1612679435074207, 0.14854630524019552, 0.12259385086810495, 0.055672518493140664, 0.02813799078937989, 0.004019967434655434]
 [99.97491934942695, 8.13631145149135, 7.9045359058287215, 7.847918834879345, 7.740466115796855, 7.5961004410520765, 7.525293130795524, 7.422301114226826, 7.27890416009798, 7.154623741055871  …  0.287640203009877, 0.236724118025973, 0.22617107993934096, 0.17734897787737688, 0.16405997475114945, 0.10738905216166734, 0.09961959432527073, 0.06895712121746406, 0.041138601551209464, 0.019051549507549753]
 [100.54302411474775, 8.06627070102437, 7.816097994710592, 7.740430872496167, 7.672445958918696, 7.6010810201107954, 7.497556443647827, 7.407289998201957, 7.36

Let's check that this indeed utilized multiple workers.

In [116]:
pmap(M) do m
    println(myid());
    svdvals(m)
end;

      From worker 6:	6
      From worker 9:	9
      From worker 8:	8
      From worker 7:	7
      From worker 6:	6
      From worker 9:	9
      From worker 8:	8
      From worker 7:	7
      From worker 6:	6
      From worker 9:	9


In [119]:
@btime map($svdvals, $M);
@btime pmap($svdvals, $M);

  54.712 ms (81 allocations: 4.22 MiB)
  15.392 ms (519 allocations: 37.59 KiB)


### When to choose which? (`@distributed` vs `pmap`)

Julia's `pmap` is designed for the case where

* one wants to apply **a function to a collection**,
* each function call does a **larger amount of work**, and/or
* the **workload is non-uniform** (load-balancing).

On the other hand, `@distributed` is good for

* **reductions**, like sums, where
* **each iteration may be tiny**, i.e. perhaps only summing two numbers, and/or
* each iteration **takes about the same time** (uniform)

## High-level array abstractions: [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl)

In a `DArray`, each process has local access to just a chunk of the data, and no two processes share the same chunk. Processes can be on different hosts.

Distributed arrays are for example useful if

* Expensive calculations should be performed in parallel on parts of the array on different hosts.
* The data doesn't fit into the local machines memory (i.e. loading big files in parallel).

In [133]:
using Distributed, BenchmarkTools; rmprocs(workers());

In [134]:
# make sure that all workers use the same Julia environment
addprocs(4; exeflags="--project")

4-element Vector{Int64}:
 18
 19
 20
 21

In [136]:
# check
@everywhere @show Base.active_project()

Base.active_project() = "/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/Project.toml"
      From worker 18:	Base.active_project() = "/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/Project.toml"
      From worker 20:	Base.active_project() = "/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/Project.toml"
      From worker 19:	Base.active_project() = "/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/Project.toml"
      From worker 21:	Base.active_project() = "/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/Project.toml"


In [137]:
@everywhere using DistributedArrays, LinearAlgebra

In [138]:
M = Matrix{Float64}[rand(200,200) for i = 1:10];

In [139]:
D = distribute(M)

10-element DArray{Matrix{Float64}, 1, Vector{Matrix{Float64}}}:
 [0.5237691826804336 0.19225096693554433 … 0.5090894478257938 0.4291709018910801; 0.784076589614852 0.9465938731556683 … 0.8866692692147827 0.23078868748940606; … ; 0.9301320539467216 0.18002927673847213 … 0.20712165241145475 0.38691641344246575; 0.014943227868830067 0.9594987864892671 … 0.1927723985879496 0.3766169725992071]
 [0.8136404329733 0.05781792976522582 … 0.2050434642570984 0.8446583617836078; 0.7404851199722592 0.515868618368856 … 0.3602494061828969 0.10575579930925216; … ; 0.41589684599812193 0.4130817814398151 … 0.37854001243504165 0.08852822867485644; 0.6534747412823856 0.6649896285915354 … 0.18282864782699548 0.5905227171779412]
 [0.7030968713815184 0.8070261550611442 … 0.5529152789870686 0.3656795854446987; 0.683572997346583 0.7639500141177592 … 0.8344616585922441 0.7344088914926489; … ; 0.5898838278833306 0.1080725735105097 … 0.7600821379000914 0.32532611977604753; 0.0639964176175869 0.2989436703831376 … 0

Which workers hold parts of D?

In [141]:
procs(D)

4-element Vector{Int64}:
 18
 19
 20
 21

In [142]:
workers()

4-element Vector{Int64}:
 18
 19
 20
 21

Which parts do they hold?

In [143]:
localpart(D) # the master doesn't hold anything

Matrix{Float64}[]

In [149]:
# Which parts do they hold?
for p in workers()
    println(@fetchfrom p DistributedArrays.localindices(D))
end

(1:3,)
(4:6,)
(7:8,)
(9:10,)


In [158]:
@btime map($svdvals, $M);
@btime map($svdvals, $D);

  52.903 ms (81 allocations: 4.22 MiB)
  13.749 ms (577 allocations: 30.42 KiB)


In [159]:
@btime pmap($svdvals, $M);

  15.324 ms (518 allocations: 37.33 KiB)


## *Actual* distributed computing: Comments

### Creating workers on other machines

So far we have worked with multiple process on the same system, because we simply used `addprocs(::Integer)`. To put worker processes on other machines, e.g. nodes of a cluster, we need to modify the initial `addprocs` call appropriately.

In Julia, starting worker processes is handled by [ClusterManagers](https://docs.julialang.org/en/v1/manual/distributed-computing/#ClusterManagers).

* The default one is `LocalManager`. It is automatically used when running `addprocs(i::Integer)` and we have implicitly used it already.
* Another important one is `SSHManager`. It is automatically used when running `addprocs(hostnames::Array)`, e.g. `addprocs(["node123", "node456"])`. The only requirement is a **passwordless ssh access** to all specified hosts.
* Cluster managers for SLURM, PBS, and others are provided in [ClusterManagers.jl](https://github.com/JuliaParallel/ClusterManagers.jl). For SLURM, this will make `addprocs` use `srun` under the hood.

*Demonstrate in terminal from thp node*

```julia
using Distributed

addprocs(["l93", "l94"])

@everywhere println(gethostname())
```

One can also start multiple processes on different machines:
```julia
addprocs([("node123", 2), ("node456", 3)]) # starts 2 workers on node123 and 3 workers on node456

# Use :auto to start as many processes as CPU threads are available
```

**Be aware of different paths:**
* By default, `addprocs` expects to find the julia executable on the remote machines under the same path as on the host (master).
* It will also try to `cd` to the same folder (set the working directory).


As you can see from `?addprocs`, `addprocs` takes a bunch of keyword arguments, two of which are of particular importance in this regard:

* `dir`: working directory for the worker processes
* `exename`: path to julia executable (potentially augmented with pre-commands) for the worker processes

In [163]:
# cleanup
rmprocs(workers())

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-x64-6.0/build/default-macmini-x64-6-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Distributed/src/cluster.jl:1048


Task (done) @0x000000011c56b9d0